In [ ]:
!pip install colorama
!pip install scapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scapy: filename=scapy-2.5.0-py2.py3-none-any.whl size=1444327 sha256=d54fd0205225b31672c67041be3cafb8ab70f48fc7a49a96feeb0c9dd383590d
  Stored in directory: /root/.cache/pip/wheels/82/b7/03/8344d8cf6695624746311bc0d389e9d05535ca83c35f90241d
Successfully built scapy


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True, timeout_ms=300000)

Mounted at /content/drive


In [ ]:
# packet sniffer
import scapy.all as scapy
import argparse
from scapy.layers import http
import colorama
from colorama import Fore
import csv

# Machine Learning
import math, time, random, datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import model_selection
from sklearn import metrics
plt.style.use('seaborn')
import warnings
warnings.filterwarnings('ignore')
import missingno
import pickle

# models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC

<ipython-input-3-2ec59c5da056>:17: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
default_values = {
    'id': 0, 'dur': 0, 'proto': 0, 'service': 0, 'state': 0, 'spkts': 0,
    'dpkts': 0, 'sbytes': 0, 'dbytes': 0, 'rate': 0, 'sttl': 0, 'dttl': 0,
    'sload': 0, 'dload': 0, 'sloss': 0, 'dloss': 0, 'sinpkt': 0, 'dinpkt': 0,
    'sjit': 0, 'djit': 0, 'swin': 0, 'stcpb': 0, 'dtcpb': 0, 'dwin': 0,
    'tcprtt': 0, 'synack': 0, 'ackdat': 0, 'smean': 0, 'dmean': 0,
    'trans_depth': 0, 'response_body_len': 0, 'ct_srv_src': 0,
    'ct_state_ttl': 0, 'ct_dst_ltm': 0, 'ct_src_dport_ltm': 0,
    'ct_dst_sport_ltm': 0, 'ct_dst_src_ltm': 0, 'is_ftp_login': 0,
    'ct_ftp_cmd': 0, 'ct_flw_http_mthd': 0, 'ct_src_ltm': 0, 'ct_srv_dst': 0,
    'is_sm_ips_ports': 0, 'attack_cat': "Normal", 'label': 0
}

data = []

def extract_data(packet):
    data = default_values.copy()
    def update_data(key, value):
        if key in data:
            data[key] = value

    # HTTP packets
    if packet.haslayer(http.HTTPRequest):
        update_data('proto', 'HTTP')

    # IP packets
    elif packet.haslayer(scapy.IP):
        ip_layer = packet[scapy.IP]
        update_data('sbytes', len(ip_layer))
        update_data('proto', ip_layer.proto)

    # TCP packets
    elif packet.haslayer(scapy.TCP):
        tcp_layer = packet[scapy.TCP]
        update_data('spkts', 1)
        update_data('dpkts', 0)

    # UDP packets
    elif packet.haslayer(scapy.UDP):
        udp_layer = packet[scapy.UDP]
        update_data('spkts', 1)
        update_data('dpkts', 0)

    # ICMP packets
    elif packet.haslayer(scapy.ICMP):
        icmp_layer = packet[scapy.ICMP]
        update_data('spkts', 1)
        update_data('dpkts', 0)

    return list(data.values())

def update_csv():
    global data
    csv_file_path = '/content/drive/My Drive/UNSW_NB15_training-set.csv'
    with open(csv_file_path, mode='r') as file:
        reader = list(csv.reader(file))

    max_id = max(int(row[0]) for row in reader[1:]) if len(reader) > 1 else 0
    for i, row in enumerate(data):
        row[0] = max_id + i + 1

    reader.extend(data)

    with open(csv_file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(reader)

def get_interface():
    parser = argparse.ArgumentParser()
    parser.add_argument('-f')
    parser.add_argument("-i", "--interface", dest="interface", help="Specify the network interface")
    arguments = parser.parse_args()
    return arguments.interface

def sniff(iface):
    global data
    packet_count = 0

    def process_packet(packet):
        nonlocal packet_count
        print(f"[+] Packet {packet_count + 1} captured")
        packet.show()
        extracted_data = extract_data(packet)
        data.append(extracted_data)
        packet_count += 1

    scapy.sniff(iface=iface, store=False, prn=process_packet, stop_filter=lambda _: packet_count >= 10)

iface = get_interface()
sniff(iface)
update_csv()

[+] Packet 1 captured
###[ Ethernet ]### 
  dst       = 02:42:ac:1c:00:0c
  src       = 02:42:03:f0:50:b6
  type      = IPv4
###[ IP ]### 
     version   = 4
     ihl       = 5
     tos       = 0x0
     len       = 542
     id        = 13877
     flags     = DF
     frag      = 0
     ttl       = 64
     proto     = 6
     chksum    = 0xaa5f
     src       = 172.28.0.1
     dst       = 172.28.0.12
     \options   \
###[ TCP ]### 
        sport     = 46946
        dport     = 8080
        seq       = 131969586
        ack       = 898551871
        dataofs   = 8
        reserved  = 0
        flags     = PA
        window    = 249
        chksum    = 0x5a56
        urgptr    = 0
        options   = [('NOP', None), ('NOP', None), ('Timestamp', (1592315165, 124367911))]
###[ HTTP 1 ]### 
###[ HTTP Request ]### 
           Method    = 'GET'
           Path      = '/socket.io/?EIO=3&sid=TI-WrgwcNwHeQJpsAAAA&t=O_opncI&transport=polling'
           Http_Version= 'HTTP/1.1'
           A_IM      

In [ ]:
train_data = pd.read_csv('/content/drive/My Drive/UNSW_NB15_training-set.csv')
test_data = pd.read_csv('/content/drive/My Drive/UNSW_NB15_testing-set.csv')

# Combine the data to avoid preprocessing twice
combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)

categorical_cols = combined_data.select_dtypes('object').columns
numeric_cols = combined_data._get_numeric_data().columns
print(numeric_cols)

Index(['id', 'dur', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'sttl',
       'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sinpkt', 'dinpkt', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack', 'ackdat',
       'smean', 'dmean', 'trans_depth', 'response_body_len', 'ct_srv_src',
       'ct_state_ttl', 'ct_dst_ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm',
       'ct_dst_src_ltm', 'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd',
       'ct_src_ltm', 'ct_srv_dst', 'is_sm_ips_ports', 'label'],
      dtype='object')


In [ ]:
# Handle type of service that is '-'
combined_data['service'].unique()
combined_data['service'] = np.where(combined_data['service'] == '-', 'None', combined_data['service'])
print(combined_data['service'].unique())

# Automate the process of removing dump values
def remove_dump_values(data, columns):
    for column in columns:
        data[column] = np.where(data[column] == '-', 'None', data[column])
    return data

columns = combined_data.columns
processed_data = remove_dump_values(combined_data, columns)

['None' 'http' 'ftp' 'ftp-data' 'smtp' 'pop3' 'dns' 'snmp' 'ssl' 'dhcp'
 'irc' 'radius' 'ssh' '0']


In [ ]:
# Remove features that are not needed
processed_data = processed_data.drop(['id'], axis=1)
# Replace the attack_cat with true
processed_data.drop(['attack_cat'], axis=1, inplace=True)
categorical_cols = categorical_cols.drop(['attack_cat'])

# Perform one-hot encoding
processed_data_encoded = pd.get_dummies(processed_data, columns=categorical_cols)

In [ ]:
# Normalization
numeric_cols = list(numeric_cols)
numeric_cols.remove('label')
numeric_cols.remove('id')

processed_data_encoded[numeric_cols] = processed_data_encoded[numeric_cols].astype('float')

processed_data_encoded[numeric_cols] = (processed_data_encoded[numeric_cols] - np.min(processed_data_encoded[numeric_cols])) / np.std(processed_data_encoded[numeric_cols])

processed_data_encoded.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,state_CLO,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,1.841324e-06,0.014712,0.0,0.002854,0.0,0.566984,2.478037,0.0,0.971150,0.0,...,False,False,False,False,True,False,False,False,False,False
1,1.339145e-06,0.014712,0.0,0.010140,0.0,0.779603,2.478037,0.0,4.743658,0.0,...,False,False,False,False,True,False,False,False,False,False
2,8.369656e-07,0.014712,0.0,0.006146,0.0,1.247365,2.478037,0.0,4.600433,0.0,...,False,False,False,False,True,False,False,False,False,False
3,1.004359e-06,0.014712,0.0,0.005179,0.0,1.039471,2.478037,0.0,3.230641,0.0,...,False,False,False,False,True,False,False,False,False,False
4,1.673931e-06,0.014712,0.0,0.012235,0.0,0.623682,2.478037,0.0,4.578896,0.0,...,False,False,False,False,True,False,False,False,False,False


### LLM SETUP AND FINE TUNE USING UNSLOTH

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! Llama 3 is up to 8k
dtype = None
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 70b also works (just change the model name)
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.




---



In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
import pandas as pd

dataset_finetune = pd.DataFrame()

# Set a constant instruction
instruction_text = "this is the input of a single packet in the network classify it as safe or not where 0 is safe and 1 is not"

# Using a list filled with the constant instruction repeated for each row in the original DataFrame
dataset_finetune['instruction'] = [instruction_text] * len(processed_data_encoded)

# Creating 'input' column by concatenating column names and their values for each row
dataset_finetune['input'] = processed_data_encoded.apply(lambda row: ', '.join([f"column {col}: {row[col]}" for col in processed_data_encoded.columns[:-1]]), axis=1)

# Assuming the label is in the last column with name 'Label'a
dataset_finetune['output'] = processed_data_encoded['label']

# You now have the new dataset as required
print(dataset_finetune.head())  # Display the first few rows to check

# Optionally, you can save this new DataFrame to a CSV file
dataset_finetune.to_csv("/content/dataset_finetune.csv", index=False)

                                         instruction  \
0  this is the input of a single packet in the ne...   
1  this is the input of a single packet in the ne...   
2  this is the input of a single packet in the ne...   
3  this is the input of a single packet in the ne...   
4  this is the input of a single packet in the ne...   

                                               input output  
0  column dur: 1.8413243122757564e-06, column spk...      0  
1  column dur: 1.3391449543823683e-06, column spk...      0  
2  column dur: 8.369655964889803e-07, column spkt...      0  
3  column dur: 1.0043587157867762e-06, column spk...      0  
4  column dur: 1.6739311929779606e-06, column spk...      0  


In [ ]:
# this is basically the system prompt
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # do not forget this part!
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN # without this token generation goes on forever!
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")

dataset = load_dataset("csv", data_files="/content/dataset_finetune.csv", split='train')

# Split the dataset into train and test sets
dataset_train = dataset.train_test_split(test_size=0.8)["train"] # Adjust test_size as needed

dataset_train = dataset_train.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/51540 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 16, # increase this to make the model learn "better"
        num_train_epochs=2,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2):   0%|          | 0/51540 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.605 GB of memory reserved.


In [ ]:
# We're now kicking off the actual training of our model, which will spit out some statistics showing us how well it learns
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 51,540 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 16
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,1.354000
2,1.345000
3,1.411100
4,1.202400
5,1.073100
6,1.017900
7,0.986100
8,0.897000
9,0.846800
10,0.831400


In [ ]:
FastLanguageModel.for_inference(model)
from random import choice

random_row = choice(dataset_finetune.to_dict('records'))

inputs = tokenizer(
[
    alpaca_prompt.format(
        random_row['instruction'],  # Use the instruction from the random row
        random_row['input'],  # Use the input from the random row
        "",  # Output is left blank for generation
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
this is the input of a single packet in the network classify it as safe or not where 0 is safe and 1 is not

### Input:
column dur: 0.0014869530787223223, column spkts: 0.2501119201039609, column dpkts: 0.32148768038762454, column sbytes: 0.012833538015140847, column dbytes: 0.09234521719411545, column rate: 0.04844544049189092, column sttl: 0.30243754303590675, column dttl: 0.25718555012479627, column sload: 0.010498462031509798, column dload: 4.900152264172459, column sloss: 0.10675446095304295, column dloss: 0.2048453708003064, column sinpkt: 3.753216017605722e-05, column dinpkt: 0.00021716142917951433, column sjit: 0.00033432192782464833, column djit: 0.0039618498827539, column swin: 2.002095438780311, column stcpb: 0.23621407551256957, column dtcpb: 0.2371025394511592, column dwin: 2.0042548